In [1]:
# Clone the entire repo.
!git clone -l -s https://github.com/icassava/fgvcx-icassava.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 9433, done.
remote: Total 9433 (delta 0), reused 0 (delta 0), pack-reused 9433
Receiving objects: 100% (9433/9433), 1.30 GiB | 49.65 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (9443/9443), done.
/content/cloned-repo
cassava-banner.png     cmd-severity.png       imgs	 README.md
cassava-leaf-imgs.png  data		      LICENSE
cbsd-severity.png      disease-incidence.png  pdata.zip


In [ ]:
! unzip /content/cloned-repo/pdata.zip -d dataset
!ls dataset/

Archive:  /content/cloned-repo/pdata.zip
   creating: dataset/pdata/
   creating: dataset/pdata/test/
   creating: dataset/pdata/test/cgm/
  inflating: dataset/pdata/test/cgm/cgm.18.jpg  
  inflating: dataset/pdata/test/cgm/cgm.6.jpg  
  inflating: dataset/pdata/test/cgm/cgm.7.jpg  
  inflating: dataset/pdata/test/cgm/cgm.5.jpg  
  inflating: dataset/pdata/test/cgm/cgm.4.jpg  
  inflating: dataset/pdata/test/cgm/cgm.0.jpg  
  inflating: dataset/pdata/test/cgm/cgm.1.jpg  
  inflating: dataset/pdata/test/cgm/cgm.3.jpg  
  inflating: dataset/pdata/test/cgm/cgm.2.jpg  
  inflating: dataset/pdata/test/cgm/cgm.9.jpg  
  inflating: dataset/pdata/test/cgm/cgm.8.jpg  
  inflating: dataset/pdata/test/cgm/cgm.15.jpg  
  inflating: dataset/pdata/test/cgm/cgm.14.jpg  
  inflating: dataset/pdata/test/cgm/cgm.16.jpg  
  inflating: dataset/pdata/test/cgm/cgm.17.jpg  
  inflating: dataset/pdata/test/cgm/cgm.13.jpg  
  inflating: dataset/pdata/test/cgm/cgm.12.jpg  
  inflating: dataset/pdata/test/cgm/cg

In [ ]:
!nvidia-smi


Fri Feb  5 06:19:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 41kB/s 


In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/cloned-repo/dataset/pdata/train'
valid_path = '/content/cloned-repo/dataset/pdata/validation'

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:

# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/cloned-repo/dataset/pdata/train/*')

In [ ]:
folders

['/content/cloned-repo/dataset/pdata/train/cgm',
 '/content/cloned-repo/dataset/pdata/train/cbsd',
 '/content/cloned-repo/dataset/pdata/train/healthy',
 '/content/cloned-repo/dataset/pdata/train/cmd',
 '/content/cloned-repo/dataset/pdata/train/cbb']

In [ ]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/cloned-repo/dataset/pdata/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 268 images belonging to 5 classes.


In [ ]:
labels = (training_set.class_indices)
print(labels)
#{'Tomato___Bacterial_spot': 0, 'Tomato___Early_blight': 1, 'Tomato___Late_blight': 2, 'Tomato___Leaf_Mold': 3, 'Tomato___Septoria_leaf_spot': 4, 'Tomato___Spider_mites Two-spotted_spider_mite': 5, 'Tomato___Target_Spot': 6, 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato___Tomato_mosaic_virus': 8, 'Tomato___healthy': 9}

{'cbb': 0, 'cbsd': 1, 'cgm': 2, 'cmd': 3, 'healthy': 4}


In [ ]:
test_set = test_datagen.flow_from_directory('/content/cloned-repo/dataset/pdata/validation',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 163 images belonging to 5 classes.


In [ ]:
labels = (test_set.class_indices)
print(labels)
#{'Tomato___Bacterial_spot': 0, 'Tomato___Early_blight': 1, 'Tomato___Late_blight': 2, 'Tomato___Leaf_Mold': 3, 'Tomato___Septoria_leaf_spot': 4, 'Tomato___Spider_mites Two-spotted_spider_mite': 5, 'Tomato___Target_Spot': 6, 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato___Tomato_mosaic_virus': 8, 'Tomato___healthy': 9}

{'cbb': 0, 'cbsd': 1, 'cgm': 2, 'cmd': 3, 'healthy': 4}


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
9/9 [==============================] - 40s 4s/step - loss: 10.7876 - accuracy: 0.3025 - val_loss: 9.6457 - val_accuracy: 0.5276
Epoch 2/10
9/9 [==============================] - 35s 4s/step - loss: 7.0727 - accuracy: 0.4949 - val_loss: 2.2752 - val_accuracy: 0.6687
Epoch 3/10
9/9 [==============================] - 35s 4s/step - loss: 2.0924 - accuracy: 0.7296 - val_loss: 2.9622 - val_accuracy: 0.6564
Epoch 4/10
9/9 [==============================] - 35s 4s/step - loss: 0.6205 - accuracy: 0.8993 - val_loss: 1.4304 - val_accuracy: 0.7239
Epoch 5/10
9/9 [==============================] - 35s 4s/step - loss: 0.2452 - accuracy: 0.9326 - val_loss: 1.9340 - val_accuracy: 0.7669
Epoch 6/10
9/9 [==============================] - 36s 4s/step - loss: 0.1053 - accuracy: 0.9740 - val_loss: 1.6678 - val_accuracy: 0.8160
Epoch 7/10
9/9 [==============================] - 36s 4s/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.4140 - val_accuracy: 0.8221
Epoch 8/10
9/9 [=================

In [ ]:
from keras.models import load_model
model.save('/content/drive/MyDrive/ResNetTomato/CassavaResNet50.h5')  